In [1]:
from google.colab import files
uploaded = files.upload()

Saving Testfile1.csv to Testfile1 (1).csv


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Testfile2.xlsx to Testfile2.xlsx


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Testfile3.xlsx to Testfile3.xlsx


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Testfile4.xlsx to Testfile4.xlsx


In [21]:

# 데이터 불러오기
df = pd.read_csv("/content/Testfile1.csv")

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import lightgbm as lgb


In [5]:
data.columns

Index(['Time(s)', 'VOLT 1', 'VOLT 2', 'VOLT 3', 'VOLT 4', 'VOLT 5', 'VOLT 6',
       'VOLT 7', 'VOLT 8', 'VOLT 9'],
      dtype='object')

In [6]:
# 이상치를 탐지하여 고장 여부 추정
outlier_threshold = 0.001
df['Fault'] = (df['VOLT 1'] < outlier_threshold) * 1

In [25]:
# 특성과 타겟 분리
X = df.drop(columns=['Fault'])
y = df['Fault']

# 학습 데이터와 테스트 데이터로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [12]:
# LightGBM 모델 학습
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'binary_logloss',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
}

In [18]:
# LightGBM 데이터셋 생성
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

# LightGBM 모델 학습
params = {
    'boosting_type': 'gbdt',            # 부스팅 방법
    'objective': 'binary',              # 목적 함수
    'metric': 'binary_logloss',         # 평가 지표
    'num_leaves': 31,                    # 최대 잎의 수
    'learning_rate': 0.05,               # 학습률
    'feature_fraction': 0.9,            # 각 트리마다 무작위로 선택되는 특성의 비율
}

gbm = lgb.train(params,
                lgb_train,
                num_boost_round=1000,          # 트리 개수를 크게 설정
                valid_sets=[lgb_train, lgb_eval])

# 모델 평가
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 20, number of negative: 944
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000184 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2550
[LightGBM] [Info] Number of data points in the train set: 964, number of used features: 10
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.020747 -> initscore=-3.854394
[LightGBM] [Info] Start training from score -3.854394
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive 

In [ ]:
                early_stopping_rounds=10,     # 조기 중단을 위한 라운드 수

In [ ]:
                verbose_eval=10             # 학습 과정 출력

In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder  # Add this line to import LabelEncoder

In [20]:
def find_max_row_diff_and_sort(df):
    # 최대 차이와 해당 인덱스를 저장할 리스트 초기화
    max_diffs = []

    # 모든 열에 대해 반복
    for col in df.columns:
        # 현재 열에 대한 행 간 차이 계산
        diff = df[col].diff().abs()

        # 최대 차이와 해당 인덱스 찾기
        max_diff = diff.max()
        #max_diff = (max_diff <= 0.5).astype(int)
        max_diff_index = diff.idxmax()

        # 리스트에 (열 이름, 최대 차이 인덱스, 최대 차이) 추가
        max_diffs.append((col, max_diff_index, max_diff))

    # DataFrame으로 변환 및 '최대 차이'로 내림차순 정렬
    max_diffs_df = pd.DataFrame(max_diffs, columns=['Column', 'Max Diff Index', 'Max Diff'])
    sorted_max_diffs_df = max_diffs_df.sort_values(by='Max Diff', ascending=False).reset_index(drop=True)



    return sorted_max_diffs_df

In [22]:
# 'VOLT'가 포함된 열만 대상으로 하는 것을 가정합니다. 다른 열을 대상으로 하려면 조건을 변경하십시오.
volt_cols = [col for col in df.columns if 'VOLT' in col]
df_volt = df[volt_cols]

# 함수 실행 및 결과 출력
sorted_max_diffs_df = find_max_row_diff_and_sort(df_volt)
print(sorted_max_diffs_df)

   Column  Max Diff Index  Max Diff
0  VOLT 1             244  1.039928
1  VOLT 4             244  0.821387
2  VOLT 5             244  0.643573
3  VOLT 6             244  0.642881
4  VOLT 8             244  0.294033
5  VOLT 7             244  0.293214
6  VOLT 9             244  0.290032
7  VOLT 3             244  0.163226
8  VOLT 2             244  0.151943


In [ ]:
# 'VOLT'가 포함된 열만 대상으로 하는 것을 가정합니다. 다른 열을 대상으로 하려면 조건을 변경하십시오.
volt_cols = [col for col in df.columns if 'VOLT' in col]
df_volt = df[volt_cols]

# 함수 실행 및 결과 출력
sorted_max_diffs_df = find_max_row_diff_and_sort(df_volt)
print(sorted_max_diffs_df)

In [24]:
# 간단한 예시로, 최대 차이값의 상위 10%를 고장으로 가정
#threshold = sorted_max_diffs_df['Max Diff'].quantile(0.9)
#df['Fault'] = (sorted_max_diffs_df['Max Diff'] >= threshold).astype(int)
# −0.05×V N−1
#df['Fault'] = (sorted_max_diffs_df['Max Diff'] > 0.5).astype(int)

# 특성 및 타겟 변수 분리
X = sorted_max_diffs_df[['Max Diff']]  # 여기서는 'max_diff'만 특성으로 사용
print(X)

X['Fault'] = (X['Max Diff'] > 0.5).astype(int)
y = X['Fault']

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X[['Max Diff']], y, test_size=0.2, random_state=42)

# 모델 훈련 및 예측
model = GradientBoostingClassifier(random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# 모델 평가
print(f'Accuracy: {accuracy_score(y_test, y_pred)}') #도대체 왜...? 정확도가 1.0이 나오지..?
#test_size를 0.2로 바꿔도 accuracy는 똑같이 1.0으로 나온다.
print(classification_report(y_test, y_pred))

   Max Diff
0  1.039928
1  0.821387
2  0.643573
3  0.642881
4  0.294033
5  0.293214
6  0.290032
7  0.163226
8  0.151943
Accuracy: 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         1
           1       1.00      1.00      1.00         1

    accuracy                           1.00         2
   macro avg       1.00      1.00      1.00         2
weighted avg       1.00      1.00      1.00         2



<ipython-input-24-0eff8a67b23d>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Fault'] = (X['Max Diff'] > 0.5).astype(int)


In [26]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
import pandas as pd

In [28]:
# 모델 초기화
model = GradientBoostingClassifier()

# 하이퍼파라미터 그리드 정의
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'min_samples_split': [2, 4, 6],
    'min_samples_leaf': [1, 2, 3],
    'learning_rate': [0.01, 0.1, 0.2]
}

# 그리드 검색 초기화
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy')

# 그리드 검색 수행
grid_search.fit(X, y)

# 최적의 하이퍼파라미터 출력
print("Best Hyperparameters::\n{}".format(grid_search.best_params_))

#n_estimators: 생성할 트리의 수, max_depth : 각 트리의 최대 깊이, min_samples_split : 노드를 분할하기 위한 최소 샘플 수

Best Hyperparameters::
{'learning_rate': 0.01, 'max_depth': 3, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}


In [ ]:
#min_samples_leaf : 리프 노드에 필요한 최소 샘플 수, learning_rate : 각 트리의 기여도

In [ ]:
# 비슷한 논리로, maximum 차이를 구한 후

In [29]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier
import pandas as pd
import numpy as np


In [30]:

# 하이퍼파라미터 분포 정의
param_dist = {
    'n_estimators': np.arange(100, 500),
    'max_depth': np.arange(1, 10),
    'min_samples_split': np.arange(2, 10),
    'min_samples_leaf': np.arange(1, 10),
    'learning_rate': np.linspace(0.01, 0.2, num=20)
}

# 랜덤 검색 초기화
random_search = RandomizedSearchCV(model, param_dist, cv=5, scoring='accuracy', n_iter=100, random_state=0)

# 랜덤 검색 수행
random_search.fit(X, y)

# 최적의 하이퍼파라미터 출력
print("Best Hyperparameters::\n{}".format(random_search.best_params_))


Best Hyperparameters::
{'n_estimators': 115, 'min_samples_split': 4, 'min_samples_leaf': 3, 'max_depth': 4, 'learning_rate': 0.18000000000000002}
